<a href="https://colab.research.google.com/github/DigdinLogistico/API-1-FATEC/blob/main/SPRINT_2_CSV_TRATADO_EST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

caminho = '/content/drive/My Drive/DATA/COMEXSTAT/'

Mounted at /content/drive


In [ ]:
# Função auxiliar para carregar múltiplos CSVs rapidamente
def carregar_arquivos(nomes, prefixo=""):
    return {
        nome.lower(): pd.read_csv(f"{caminho}{prefixo}{nome.upper()}.csv", sep=";", encoding="latin1")
        for nome in nomes
    }


In [ ]:
# Arquivos para tratamento dados do estado
arquivos_estado = ["exp_2023", "exp_2024", "exp_2025", "imp_2023", "imp_2024", "imp_2025"]
arquivos_auxiliares = ["ncm", "pais", "via", "ncm_unidade", "urf"]

# Carregar os dados de estado
dados_estado = carregar_arquivos(arquivos_estado)

# Carregar dados auxiliares
dados_auxiliares = carregar_arquivos(arquivos_auxiliares)

# Verificar as primeiras linhas dos dados carregados
print(dados_estado["exp_2023"].head())


   CO_ANO  CO_MES    CO_NCM  CO_UNID  CO_PAIS SG_UF_NCM  CO_VIA   CO_URF  \
0    2023       3  84139190       10      160        RJ       4   817700   
1    2023      12  73182100       10      764        SP       1   817800   
2    2023       4  84219999       10       40        RS       4   817600   
3    2023       5  85291090       10       63        RJ       7  1017503   
4    2023      10  84825010       11       63        MG       7  1017503   

   QT_ESTAT  KG_LIQUIDO  VL_FOB  
0    107365           5      20  
1         0           0      84  
2         2           2     758  
3         1           1     275  
4         2           0       9  


In [ ]:
# Função para processar os dados de estado
def processar_estado(dados_estado, dados_auxiliares):
    df_estado = pd.concat([dados_estado[f"exp_{ano}"] for ano in range(2023, 2026)], ignore_index=True)
    df_estado = pd.concat([df_estado, pd.concat([dados_estado[f"imp_{ano}"] for ano in range(2023, 2026)], ignore_index=True)], ignore_index=True)

    # Renomear colunas conforme o dicionário
    renomear_colunas_estado = {
        "CO_ANO":"Ano",
        "CO_MES":"Mês",
        "CO_NCM":"Produto",
        "CO_UNID":"Unidade",
        "CO_PAIS":"País",
        "SG_UF_NCM":"Estado",
        "CO_VIA":"Modal",
        "CO_URF":"Aduana",
        "QT_ESTAT":"Quantidade",
        "KG_LIQUIDO":"Peso",
        "VL_FOB":"Valor US$",
    }
    df_estado = df_estado.rename(columns=renomear_colunas_estado)

    df_estado = df_estado.merge(dados_auxiliares["pais"][["CO_PAIS", "NO_PAIS"]].rename(columns={"CO_PAIS": "País", "NO_PAIS": "Nome do País"}), on="País", how="left")
    df_estado = df_estado.merge(dados_auxiliares["ncm"][["CO_NCM", "NO_NCM_POR"]].rename(columns={"CO_NCM": "Produto", "NO_NCM_POR": "Nome do Produto"}), on="Produto", how="left")
    df_estado = df_estado.merge(dados_auxiliares["via"][["CO_VIA", "NO_VIA"]].rename(columns={"CO_VIA": "Modal", "NO_VIA": "Nome do Modal"}), on="Modal", how="left")
    df_estado = df_estado.merge(dados_auxiliares["ncm_unidade"][["CO_UNID", "NO_UNID"]].rename(columns={"CO_UNID": "Unidade", "NO_UNID": "Nome da Unidade"}), on="Unidade", how="left")

    return df_estado

# Processar estado
df_estado_processado = processar_estado(dados_estado, dados_auxiliares)

# Salvar arquivo processado
df_estado_processado.to_csv('/content/drive/My Drive/DATA/COMEXSTAT/estado_processado_3.1.csv', index=False)

In [ ]:
# Visualização do data frame
print(df_estado_processado.head())

    Ano  Mês   Produto  Unidade  País Estado  Modal   Aduana  Quantidade  \
0  2023    3  84139190       10   160     RJ      4   817700      107365   
1  2023   12  73182100       10   764     SP      1   817800           0   
2  2023    4  84219999       10    40     RS      4   817600           2   
3  2023    5  85291090       10    63     RJ      7  1017503           1   
4  2023   10  84825010       11    63     MG      7  1017503           2   

   Peso  Valor US$  VL_FRETE  VL_SEGURO Nome do País  \
0     5         20       NaN        NaN        China   
1     0         84       NaN        NaN       Suécia   
2     2        758       NaN        NaN       Angola   
3     1        275       NaN        NaN    Argentina   
4     0          9       NaN        NaN    Argentina   

                                     Nome do Produto Nome do Modal  \
0              Outras partes de bombas para líquidos         AEREA   
1  Arruelas de pressão e outras arruelas de segur...      MARITIMA

In [ ]:
# Exclusão de colunas específicas
colunas_para_excluir = ["Produto", "Unidade", "País", "Modal", "Aduana", "VL_SEGURO", "VL_FRETE"]
df_estado_processado = df_estado_processado.drop(columns=colunas_para_excluir, errors='ignore')

# Relacionar os números com seus respectivos meses.
mapa_mensal = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}
df_estado_processado['Mês'] = df_estado_processado['Mês'].map(mapa_mensal)

# Visualização do DataFrame editado
display(df_estado_processado.head())

,Ano,Mês,Estado,Quantidade,Peso,Valor US$,Nome do País,Nome do Produto,Nome do Modal,Nome da Unidade
0,2023,Março,RJ,107365,5,20,China,Outras partes de bombas para líquidos,AEREA,QUILOGRAMA LIQUIDO
1,2023,Dezembro,SP,0,0,84,Suécia,Arruelas de pressão e outras arruelas de segur...,MARITIMA,QUILOGRAMA LIQUIDO
2,2023,Abril,RS,2,2,758,Angola,Outras partes de aparelhos para filtrar ou dep...,AEREA,QUILOGRAMA LIQUIDO
3,2023,Maio,RJ,1,1,275,Argentina,"Outras antenas e refletores de antenas, e suas...",RODOVIARIA,QUILOGRAMA LIQUIDO
4,2023,Outubro,MG,2,0,9,Argentina,"Rolamentos de roletes cilíndricos, de carga ra...",RODOVIARIA,NUMERO (UNIDADE)


In [ ]:
# Salvar o DataFrame finalizado em CSV
df_estado_processado.to_csv('/content/drive/My Drive/DATA/COMEXSTAT/estado_processado_3.3.csv', index=False)